#### Importing libraries

In [66]:
import pandas as pd
import numpy  as np
import seaborn as sns
from scipy import stats


#### Uploading and preparing data

In [67]:
df = pd.read_csv('/Users/igorrudakov/Desktop/CRM_Senior Data Analyst/cs_data.csv')
df = df.rename(columns={'Unnamed: 0':'local_id'})
df.head()

,local_id,analytical_customer_id,voucher_group,segment_frequency_rest,segment_monetary_rest,segment_discount_dh_rest,segment_variety_rest,segment_basket_rest,lt_order_cluster,recency_cluster,lt_cluster,times_in_churn_segments,preferred_order_period_all_verts,loyalty_status_all_verts,returning_probability_segments,orders,margin,order_x_margin
0,5,00004b56-d951-4693-aa69-d1986cac9ae0,lv-2|3|18,B,M,N,H,2,5+,90-120,720+,4+,multi weekday | multi daytime,Inactive Loyal,0.7-1,0,0.00,0.00
1,11,00008a4a-0e9a-4c05-8cef-01feaf7d5254,hv-2|10|25,E,L,N,N,1,5+,120-180,360-720,4+,single weekday | single daytime,Inactive Loyal,0.5-0.7,2,7.91,15.82
2,24,000147e2-b93d-4ca8-bd05-25014e0f7874,Control Group,E,M,H,N,1,5+,120-180,720+,4+,single weekday | single daytime,Inactive Loyal,0.5-0.7,11,1.06,11.66
3,29,000172b4-f65e-43f2-b401-02f9c377c892,hv-1|10|13,B,L,N,H,2,5+,30-60,<180,2-3,multi weekday | multi daytime,Inactive Multiple Orders,0.7-1,1,5.88,5.88
4,32,0001b27c-9814-4ec4-8eb9-2465f82b0c82,lv-1|3|9,F,L,N,N,1,1,90-120,<180,1,single weekday | single daytime,Inactive Churning 1 Timer,0.3-0.5,0,0.00,0.00


In [309]:
# Introducing new columns

df['order_x_margin'] = df.orders * df.margin
df['segment_basket_rest'] = df.segment_basket_rest.apply(lambda x: str(x))
df['treatment_flg'] = df.voucher_group.apply(lambda x: 'control' if x == 'Control Group' else 'treatment')

#### Answering the questions

##### What is the best voucher overall? 

In [389]:
cg_g_rows = df[(df.voucher_group =='Control Group')]
tg_g_rows = df[(df.voucher_group !='Control Group')]

In [391]:
tg_g_rows.dtypes

local_id                              int64
analytical_customer_id               object
voucher_group                        object
segment_frequency_rest               object
segment_monetary_rest                object
segment_discount_dh_rest             object
segment_variety_rest                 object
segment_basket_rest                  object
lt_order_cluster                     object
recency_cluster                      object
lt_cluster                           object
times_in_churn_segments              object
preferred_order_period_all_verts     object
loyalty_status_all_verts             object
returning_probability_segments       object
orders                                int64
margin                              float64
order_x_margin                      float64
treatment_flg                        object
dtype: object

In [70]:
# test for variance equality
stats.bartlett(cg_g_rows.orders, tg_g_rows.orders )

BartlettResult(statistic=8.34160420897968, pvalue=0.0038747369500419407)

In [71]:
stats.mannwhitneyu(cg_g_rows.orders,tg_g_rows.orders)

MannwhitneyuResult(statistic=6884119960.5, pvalue=8.073884560386203e-32)

In [312]:
vg_groups      = []
mannwhitneyu_p = []
metrics = []
uplifts = []

for vg in tg_g_rows.voucher_group.unique():
    for metric in ['orders','margin','order_x_margin']:
        vg_groups.append(vg)
        metrics.append(metric)
        mannwhitneyu_p.append(stats.mannwhitneyu(tg_g_rows[(tg_g_rows.voucher_group == vg)][f'{metric}'], cg_g_rows[f'{metric}'] ).pvalue)
        uplifts.append(tg_g_rows[(tg_g_rows.voucher_group == vg)][f'{metric}'].mean() / cg_g_rows[f'{metric}'].mean())
        
vs_global_cg = pd.DataFrame({'voucher_group' :vg_groups,
                             'metric':metrics,
                             'uplift':uplifts,
                              'mannwhitneyu_p_value': mannwhitneyu_p
               })

In [313]:
vs_global_cg[(vs_global_cg.mannwhitneyu_p_value < 0.05)]\
                .pivot(columns='metric', index='voucher_group', values='uplift')\
                .reset_index()\
                .sort_values(by='margin', ascending=False)

metric,voucher_group,margin,order_x_margin,orders
5,lv-3|3|27,1.078630,1.097826,1.066018
3,lv-1|3|9,0.999552,0.930017,1.047894
1,hv-2|10|25,0.993163,1.095297,1.124527
6,mv-1|6|10,0.939460,0.885700,1.097767
0,hv-1|10|13,0.893663,1.010762,1.171705
2,hv-3|10|38,NaN,NaN,1.042011
4,lv-2|3|18,NaN,NaN,1.016413
7,mv-2|6|20,NaN,NaN,1.057131
8,mv-3|6|30,NaN,NaN,1.034492


Best overall  : lv-3|3|27  - maximizes orders and margin (+7.8% margin, +6.6% orders)
Best by orders: hv-1|10|13 - maximizes orders but drops margin considerably (+17% orders, -10.6% margin)

In [314]:
vs_global_cg.sort_values(by='mannwhitneyu_p_value').head(15)

,voucher_group,metric,uplift,mannwhitneyu_p_value
7,hv-1|10|13,margin,0.893663,1.005569e-39
8,hv-1|10|13,order_x_margin,1.010762,1.257109e-36
6,hv-1|10|13,orders,1.171705,9.179383e-36
21,mv-1|6|10,orders,1.097767,8.044949e-19
3,hv-2|10|25,orders,1.124527,4.109420e-14
9,lv-1|3|9,orders,1.047894,8.058988e-07
22,mv-1|6|10,margin,0.939460,1.965303e-06
15,mv-2|6|20,orders,1.057131,2.634346e-06
23,mv-1|6|10,order_x_margin,0.885700,1.581412e-05
4,hv-2|10|25,margin,0.993163,8.326602e-05


In [ ]:
with the least p value
# orders
# hv-1|10|13 
# mv-1|6|10

# order and margin
# lv-3|3|27

##### What is the best single dimensions segmentation to be used? 

,local_id,analytical_customer_id,voucher_group,segment_frequency_rest,segment_monetary_rest,segment_discount_dh_rest,segment_variety_rest,segment_basket_rest,lt_order_cluster,recency_cluster,lt_cluster,times_in_churn_segments,preferred_order_period_all_verts,loyalty_status_all_verts,returning_probability_segments,orders,margin
0,5,00004b56-d951-4693-aa69-d1986cac9ae0,lv-2|3|18,B,M,N,H,2,5+,90-120,720+,4+,multi weekday | multi daytime,Inactive Loyal,0.7-1,0,0.00
1,11,00008a4a-0e9a-4c05-8cef-01feaf7d5254,hv-2|10|25,E,L,N,N,1,5+,120-180,360-720,4+,single weekday | single daytime,Inactive Loyal,0.5-0.7,2,7.91
2,24,000147e2-b93d-4ca8-bd05-25014e0f7874,Control Group,E,M,H,N,1,5+,120-180,720+,4+,single weekday | single daytime,Inactive Loyal,0.5-0.7,11,1.06
3,29,000172b4-f65e-43f2-b401-02f9c377c892,hv-1|10|13,B,L,N,H,2,5+,30-60,<180,2-3,multi weekday | multi daytime,Inactive Multiple Orders,0.7-1,1,5.88
4,32,0001b27c-9814-4ec4-8eb9-2465f82b0c82,lv-1|3|9,F,L,N,N,1,1,90-120,<180,1,single weekday | single daytime,Inactive Churning 1 Timer,0.3-0.5,0,0.00


In [340]:
# all segmentation columns
segmentation_approaches = [
                             'segment_frequency_rest'
                            ,'segment_monetary_rest'
                            ,'segment_discount_dh_rest'
                            ,'segment_variety_rest'
                            ,'segment_basket_rest'
                            ,'lt_order_cluster'
                            ,'recency_cluster'
                            ,'lt_cluster'
                            ,'times_in_churn_segments'
                            ,'preferred_order_period_all_verts'
                            ,'loyalty_status_all_verts'
                            ,'returning_probability_segments']

In [374]:
# Get distributions of orders by control and treatment
# run a Chi-squared test

segmentation_nm = []
chi_squared_results = []

for segmentation_approach in segmentation_approaches:
    segmentation_nm.append(segmentation_approach)
    a = df.groupby([segmentation_approach,'treatment_flg'], as_index=False).agg(
        orders_sum=('orders', np.sum)
    ).pivot(index=segmentation_approach, columns='treatment_flg', values='orders_sum').reset_index()
    chi_squared_results.append(stats.chisquare(a.control, a.treatment).pvalue)

chi_squared_for_segments = pd.DataFrame({'seg_group':segmentation_nm,
                                         'chi_squared_p_value':chi_squared_results
                                       })
chi_squared_for_segments

,seg_group,user_count_tg
0,segment_frequency_rest,0.000000e+00
1,segment_monetary_rest,2.476678e-288
2,segment_discount_dh_rest,0.000000e+00
3,segment_variety_rest,3.216904e-301
4,segment_basket_rest,6.044300e-298
5,lt_order_cluster,0.000000e+00
6,recency_cluster,0.000000e+00
7,lt_cluster,1.742193e-296
8,times_in_churn_segments,0.000000e+00
9,preferred_order_period_all_verts,2.061196e-299


In [375]:
chi_squared_for_segments[(chi_squared_for_segments.user_count_tg > 0.05)]

,seg_group,user_count_tg


In [393]:
seg_group        = []
bucket_group     = []
uplifts_seg      = []
uplifts          = []
mannwhitneyu_seg = []
mannwhitneyu_vg  = []
vg_groups        = []
user_count       = []

for segmentation_approach in segmentation_approaches:
    for bucket in cg_g_rows[f'{segmentation_approach}'].unique():
        for vg in tg_g_rows.voucher_group.unique():
            vg_groups.append(vg)
            seg_group.append(segmentation_approach)
            bucket_group.append(bucket)
            
#      uplift vs contol group
            uplifts_seg.append(
                tg_g_rows[(tg_g_rows[f'{segmentation_approach}']==f'{bucket}')].margin.mean()/   \
                cg_g_rows[(cg_g_rows[f'{segmentation_approach}']==f'{bucket}')].margin.mean()    # control
            )
    
            user_count.append(
                tg_g_rows[(tg_g_rows[f'{segmentation_approach}']==f'{bucket}')].orders.sum()
            )
            
            uplifts.append(
                tg_g_rows[(tg_g_rows[f'{segmentation_approach}']==f'{bucket}') & (tg_g_rows.voucher_group == vg)].margin.mean()/\
                cg_g_rows[(cg_g_rows[f'{segmentation_approach}']==f'{bucket}')                                  ].margin.mean() )  # control

            mannwhitneyu_seg.append(stats.mannwhitneyu(
                                    tg_g_rows[(tg_g_rows[f'{segmentation_approach}']==f'{bucket}')].margin,\
                                    cg_g_rows[(cg_g_rows[f'{segmentation_approach}']==f'{bucket}')].margin
                                ).pvalue < 0.05
                             )
            
            mannwhitneyu_vg.append(stats.mannwhitneyu(
                                tg_g_rows[(tg_g_rows[f'{segmentation_approach}']==f'{bucket}') & (tg_g_rows.voucher_group == vg)].margin,\
                                cg_g_rows[(cg_g_rows[f'{segmentation_approach}']==f'{bucket}')].margin
                            ).pvalue < 0.05
                         )

seg_appr_dif = pd.DataFrame({'seg_group':seg_group,
                             'user_count_tg':user_count,
                             'bucket_group':bucket_group,
                             'vg_groups':vg_groups,
                             'uplifts_bucket':uplifts_seg,
                             'uplifts_vg':uplifts,
                             'mannwhitneyu_seg':mannwhitneyu_seg,
                             'mannwhitneyu_vg':mannwhitneyu_vg
               })

In [394]:
# next we multiply uplift for each bucket-voucher instance by the mannwhitneyu result (1/0 - significant or not)
# So, if uplift is not statistically significant we would conclude that there is no uplift

seg_appr_dif['uplift_x_pval_bucket']  = seg_appr_dif.uplifts_bucket * seg_appr_dif.mannwhitneyu_seg
seg_appr_dif['uplift_x_pval_voucher'] = seg_appr_dif.uplifts_vg     * seg_appr_dif.mannwhitneyu_vg

In [414]:
# present it in a matrix
fin = seg_appr_dif.pivot_table(index=['seg_group','bucket_group'], columns='vg_groups', values=['uplift_x_pval_voucher'])\
            .sort_values(by=['seg_group','bucket_group'])

seg_group                     bucket_group  \
vg_groups                                                                      
0                  loyalty_status_all_verts        Inactive Churning 1 Timer   
1                  loyalty_status_all_verts                   Inactive Loyal   
2                  loyalty_status_all_verts         Inactive Multiple Orders   
3                                lt_cluster                          180-360   
4                                lt_cluster                          360-720   
5                                lt_cluster                             720+   
6                                lt_cluster                             <180   
7                          lt_order_cluster                                1   
8                          lt_order_cluster                              2-4   
9                          lt_order_cluster                               5+   
10         preferred_order_period_all_verts    multi weekday | multi daytime   
11         preferred_order_period_all_verts   multi weekday | single daytime   
12         preferred_order_period_all_verts   single weekday | multi daytime   
13         preferred_order_period_all_verts  single weekday | single daytime   
14                          recency_cluster                          120-180   
15                          recency_cluster                            30-60   
16                          recency_cluster                            60-90   
17                          recency_cluster                           90-120   
18           returning_probability_segments                            0-0.3   
19           returning_probability_segments                          0.3-0.5   
20           returning_probability_segments                          0.5-0.7   
21           returning_probability_segments                            0.7-1   
22           returning_probability_segments                         No_Score   
23                      segment_basket_rest                                1   
24                      segment_basket_rest                                2   
25                      segment_basket_rest                                3   
26                 segment_discount_dh_rest                                H   
27                 segment_discount_dh_rest                                M   
28                 segment_discount_dh_rest                                N   
29                   segment_frequency_rest                                A   
30                   segment_frequency_rest                                B   
31                   segment_frequency_rest                                C   
32                   segment_frequency_rest                                D   
33                   segment_frequency_rest                                E   
34                   segment_frequency_rest                                F   
35                   segment_frequency_rest                                M   
36                    segment_monetary_rest                                H   
37                    segment_monetary_rest                                L   
38                    segment_monetary_rest                                M   
39                     segment_variety_rest                                H   
40                     segment_variety_rest                                L   
41                     segment_variety_rest                                M   
42                     segment_variety_rest                                N   
43                  times_in_churn_segments                                1   
44                  times_in_churn_segments                              2-3   
45                  times_in_churn_segments                               4+   

          uplift_x_pval_voucher                                            \
vg_groups            hv-1|10|13 hv-2|10|25 hv-3|10|38  lv-1|3|9 lv-2|3|18   
0                      0.681623   0.872

In [429]:
fin.to_excel('output.xlsx')